In [36]:
# creating documnets

sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp=[sample_text_1,sample_text_2,sample_text_3]
no_docs=len(corp)

print("Number of documents:{0}".format(no_docs))

Number of documents:3


In [43]:
# INTEGER ENCODING ALL THE DOCUMENTS

from tensorflow.keras.preprocessing.text import one_hot,Tokenizer

vocab_size = 50
encod_corp = []

for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,50))
    print("The encoding for document",i+1," is : ",one_hot(doc,50))

The encoding for document 1  is :  [48, 35, 35, 44, 37, 13]
The encoding for document 2  is :  [20, 25, 44, 37, 13, 21, 35, 44, 14]
The encoding for document 3  is :  [30, 39, 35, 17, 11, 13, 21, 42, 25, 14, 13, 11]


In [46]:
# PADDING THE DOCS(To MAKE VERY DOCS Of Same Length)

import nltk

# Get maximum number of words in any document
maxlen = -1
for doc in corp:
    tokens = nltk.word_tokenize(doc)
    if(maxlen < len(tokens)):
        maxlen = len(tokens)

print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  12


In [54]:
# Create Padded Sequence to make docs of same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='post',value=0.0)

print("No of padded documents: ",len(pad_corp))

pad_corp

No of padded documents:  3


array([[48, 35, 35, 44, 37, 13,  0,  0,  0,  0,  0,  0],
       [20, 25, 44, 37, 13, 21, 35, 44, 14,  0,  0,  0],
       [30, 39, 35, 17, 11, 13, 21, 42, 25, 14, 13, 11]])

In [63]:
# ACTUALLY CREATING THE EMBEDDINGS using KERAS EMBEDDING LAYER
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Flatten
from tensorflow.keras.optimizers import Adam


# specifying the input shape
input=Input(shape=(no_docs,maxlen),dtype='float64')

# shape of input. each document has 12 element or words which is the value of our maxlen variable.
word_input=Input(shape=(maxlen,),dtype='float64')  

# creating the embedding
word_embedding=Embedding(input_dim=vocab_size,output_dim=8,input_length=maxlen)(word_input)

word_vec=Flatten()(word_embedding) # flatten

embed_model =Model([word_input],word_vec) # combining all into a Keras mode

In [64]:
embed_model.compile(optimizer=Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc']) 

In [65]:
print(type(word_embedding))
print(word_embedding)

<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("embedding_11/embedding_lookup/Identity_1:0", shape=(?, 12, 8), dtype=float32)


In [66]:
print(embed_model.summary()) # summary of the model

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 12)]              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 12, 8)             400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 96)                0         
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________
None


In [67]:
embeddings=embed_model.predict(pad_corp) # finally getting the embeddings.

In [68]:
print("Shape of embeddings : ",embeddings.shape)
print(embeddings)

Shape of embeddings :  (3, 96)
[[-0.03358375 -0.02131583  0.03760034  0.02735627  0.04701524 -0.02539201
   0.02710209  0.03554061  0.03714552 -0.0268059   0.03889337 -0.04620111
   0.04052329 -0.01725221 -0.00124292 -0.03632836  0.03714552 -0.0268059
   0.03889337 -0.04620111  0.04052329 -0.01725221 -0.00124292 -0.03632836
   0.02784251  0.04658312  0.03572978 -0.03609236 -0.01189877  0.01943404
   0.02809929  0.01358377  0.04421194 -0.03531181 -0.03053947 -0.04904984
   0.00812716  0.01497671 -0.04858831 -0.03165916  0.0164254  -0.04280375
  -0.04880965  0.03039367 -0.0438689   0.04047049  0.00176177  0.02224557
   0.02223543 -0.02939804 -0.04749351 -0.0296113   0.04786566  0.02868212
   0.03664417  0.03842005  0.02223543 -0.02939804 -0.04749351 -0.0296113
   0.04786566  0.02868212  0.03664417  0.03842005  0.02223543 -0.02939804
  -0.04749351 -0.0296113   0.04786566  0.02868212  0.03664417  0.03842005
   0.02223543 -0.02939804 -0.04749351 -0.0296113   0.04786566  0.02868212
   0.0366

In [69]:
embeddings=embeddings.reshape(-1,maxlen,8)
print("Shape of embeddings : ",embeddings.shape) 
print(embeddings)

Shape of embeddings :  (3, 12, 8)
[[[-0.03358375 -0.02131583  0.03760034  0.02735627  0.04701524
   -0.02539201  0.02710209  0.03554061]
  [ 0.03714552 -0.0268059   0.03889337 -0.04620111  0.04052329
   -0.01725221 -0.00124292 -0.03632836]
  [ 0.03714552 -0.0268059   0.03889337 -0.04620111  0.04052329
   -0.01725221 -0.00124292 -0.03632836]
  [ 0.02784251  0.04658312  0.03572978 -0.03609236 -0.01189877
    0.01943404  0.02809929  0.01358377]
  [ 0.04421194 -0.03531181 -0.03053947 -0.04904984  0.00812716
    0.01497671 -0.04858831 -0.03165916]
  [ 0.0164254  -0.04280375 -0.04880965  0.03039367 -0.0438689
    0.04047049  0.00176177  0.02224557]
  [ 0.02223543 -0.02939804 -0.04749351 -0.0296113   0.04786566
    0.02868212  0.03664417  0.03842005]
  [ 0.02223543 -0.02939804 -0.04749351 -0.0296113   0.04786566
    0.02868212  0.03664417  0.03842005]
  [ 0.02223543 -0.02939804 -0.04749351 -0.0296113   0.04786566
    0.02868212  0.03664417  0.03842005]
  [ 0.02223543 -0.02939804 -0.04749351 -

In [70]:
for i,doc in enumerate(embeddings):
    for j,word in enumerate(doc):
        print("The encoding for ",j+1,"th word","in",i+1,"th document is : \n\n",word)

The encoding for  1 th word in 1 th document is : 

 [-0.03358375 -0.02131583  0.03760034  0.02735627  0.04701524 -0.02539201
  0.02710209  0.03554061]
The encoding for  2 th word in 1 th document is : 

 [ 0.03714552 -0.0268059   0.03889337 -0.04620111  0.04052329 -0.01725221
 -0.00124292 -0.03632836]
The encoding for  3 th word in 1 th document is : 

 [ 0.03714552 -0.0268059   0.03889337 -0.04620111  0.04052329 -0.01725221
 -0.00124292 -0.03632836]
The encoding for  4 th word in 1 th document is : 

 [ 0.02784251  0.04658312  0.03572978 -0.03609236 -0.01189877  0.01943404
  0.02809929  0.01358377]
The encoding for  5 th word in 1 th document is : 

 [ 0.04421194 -0.03531181 -0.03053947 -0.04904984  0.00812716  0.01497671
 -0.04858831 -0.03165916]
The encoding for  6 th word in 1 th document is : 

 [ 0.0164254  -0.04280375 -0.04880965  0.03039367 -0.0438689   0.04047049
  0.00176177  0.02224557]
The encoding for  7 th word in 1 th document is : 

 [ 0.02223543 -0.02939804 -0.0474935